In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi
import pickle

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/FetalBrain_AllData_AnnData_1.h5ad", "rb"))

In [3]:
adata = adata[adata.obs['Age'] == '19 pcw',:]
adata = adata[adata.obs['Pool'] == 'CRcells',:]

In [4]:
adata.obs['GeoLOD2.0_01'] = adata.obs['GeoLOD2.0_01'].astype(float)
detected = np.array([adata.X[i,:] >= adata.obs['GeoLOD2.0_01'].iloc[i] for i in range(len(adata.X[:,1]))])

Trying to set attribute `.obs` of view, copying.


In [5]:
adata.X = adata.layers['X_corrected_cc_cpm']
res = {}
count = 0
tissues = np.unique(adata.obs['Tissue'])
for i in range(len(tissues)):
    for j in range((i+1),len(tissues)):
        print(i)
        print(j)
        if i !=j:
            new_key = str(tissues[i]) + '_vs_' + str(tissues[j])
            subset_tissue_i = np.sum(detected[adata.obs['Tissue'] == tissues[i],:], axis = 0)/np.sum(adata.obs['Tissue'] == tissues[i]) > 0.8
            subset_tissue_j = np.sum(detected[adata.obs['Tissue'] == tissues[j],:], axis = 0)/np.sum(adata.obs['Tissue'] == tissues[j]) > 0.8
            print(new_key)
            adata_subset = adata[[x in (tissues[i], tissues[j]) for x in adata.obs['Tissue']],
                                 subset_tissue_i + subset_tissue_j]
            print(adata.shape)
            print(adata_subset.shape)
            res[new_key] = de.test.rank_test(
                data=adata_subset,
                grouping="Tissue",
                is_logged=False
            ).summary()
        
        tissue1 = tissues[i]
        tissue2 = tissues[j]
        
        if np.sum(np.array(res[new_key]['gene']) == np.array(adata_subset.var_names)) != len(adata_subset.var_names):
            raise ValueError()

        res[new_key]['log2_mean_' + tissue1] = np.array(np.log2(np.mean(adata_subset[adata_subset.obs['Tissue'] == tissue1,].X, axis = 0)))
        res[new_key]['log2_mean_' + tissue2] = np.array(np.log2(np.mean(adata_subset[adata_subset.obs['Tissue'] == tissue2,].X, axis = 0)))
        res[new_key]['log2_fold_change'] =  res[new_key]['log2_mean_' + tissue1] - res[new_key]['log2_mean_' + tissue2]

        subset1 = [adata_subset.obs['Tissue'].iloc[j] == tissue1 and adata_subset.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata_subset.obs['Donor']))]
        subset2 = [adata_subset.obs['Tissue'].iloc[j] == tissue2 and adata_subset.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata_subset.obs['Donor']))]
        res[new_key]['log2_mean_Hob7_' + tissue1] = np.array(np.log2(np.mean(adata_subset[subset1,].X, axis = 0)))
        res[new_key]['log2_mean_Hob7_' + tissue2] = np.array(np.log2(np.mean(adata_subset[subset2,].X, axis = 0)))
        res[new_key]['log2_fold_change_Hob7'] =  res[new_key]['log2_mean_Hob7_' + tissue1] - res[new_key]['log2_mean_Hob7_' + tissue2]

        print((adata_subset[subset1,].obs['Tissue'].astype(str) + ' ' + adata_subset[subset1,].obs['Pool'].astype(str)
     + ' ' + adata_subset[subset1,].obs['Donor'].astype(str)).value_counts())
        print((adata_subset[subset2,].obs['Tissue'].astype(str) + ' ' + adata_subset[subset2,].obs['Pool'].astype(str)
     + ' ' + adata_subset[subset2,].obs['Donor'].astype(str)).value_counts())

        subset1 = [adata_subset.obs['Tissue'].iloc[j] == tissue1 and adata_subset.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata_subset.obs['Donor']))]
        subset2 = [adata_subset.obs['Tissue'].iloc[j] == tissue2 and adata_subset.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata_subset.obs['Donor']))]
        res[new_key]['log2_mean_Hob8_' + tissue1] = np.array(np.log2(np.mean(adata_subset[subset1,].X, axis = 0)))
        res[new_key]['log2_mean_Hob8_' + tissue2] = np.array(np.log2(np.mean(adata_subset[subset2,].X, axis = 0)))
        res[new_key]['log2_fold_change_Hob8'] =  res[new_key]['log2_mean_Hob8_' + tissue1] - res[new_key]['log2_mean_Hob8_' + tissue2]

        print((adata_subset[subset1,].obs['Tissue'].astype(str) + ' ' + adata_subset[subset1,].obs['Pool'].astype(str)
     + ' ' + adata_subset[subset1,].obs['Donor'].astype(str)).value_counts())
        print((adata_subset[subset2,].obs['Tissue'].astype(str) + ' ' + adata_subset[subset2,].obs['Pool'].astype(str)
     + ' ' + adata_subset[subset2,].obs['Donor'].astype(str)).value_counts())

        res[new_key] = res[new_key].sort_values('pval')
    
res_naive = res

0
1
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
(74, 17498)
(38, 1784)
Foetal brain (frontal lobe) CRcells Hob7    6
dtype: int64
Foetal brain (occipital lobe) CRcells Hob7    12
dtype: int64
Foetal brain (frontal lobe) CRcells Hob8    12
dtype: int64
Foetal brain (occipital lobe) CRcells Hob8    8
dtype: int64
0
2
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
(74, 17498)
(34, 1321)
Foetal brain (frontal lobe) CRcells Hob7    6
dtype: int64
Foetal brain (parietal lobe) CRcells Hob7    7
dtype: int64
Foetal brain (frontal lobe) CRcells Hob8    12
dtype: int64
Foetal brain (parietal lobe) CRcells Hob8    9
dtype: int64
0
3
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
(74, 17498)
(38, 1169)
Foetal brain (frontal lobe) CRcells Hob7    6
dtype: int64
Foetal brain (temporal lobe) CRcells Hob7    10
dtype: int64
Foetal brain (frontal lobe) CRcells Hob8    12
dtype: int64
Foetal brain (temporal lobe) CRcells Hob8    10
dtype: int64
1
2
Foetal brain

Plot top genes in occipital-frontal comparison:

In [6]:
res[list(res.keys())[0]].loc[[res[list(res.keys())[0]]['qval'].iloc[i] < 0.05 and res[list(res.keys())[0]]['log2fc'].iloc[i] > 1
 for i in range(len(res[list(res.keys())[0]]['log2fc']))]].to_csv('test2.csv')

In [7]:
# pickle.dump(res_naive, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_naive_1.p", "wb" ))

Save all data on pairwise differential expression in CR cells in one data frame:

In [8]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cell_DE_results.xlsx', engine='xlsxwriter')

# # Write each dataframe to a different worksheet.
# for i in range(len(allResults)):
#     allResults[i].to_excel(writer, sheet_name=key_wta_array[i].split('Foetal brain')[1].split('lobe')[0].split('(')[1]
#                            + 'vs '
#                            + key_wta_array[i].split('Foetal brain')[2].split('lobe')[0].split('(')[1])

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()